In [1]:
import os
from datasets import load_dataset, concatenate_datasets
from tokenizers.models import WordLevel, BPE
from tokenizers import Tokenizer
from tokenizers.trainers import WordLevelTrainer, BpeTrainer
from tokenizers.pre_tokenizers import Whitespace
from tokenizers.processors import TemplateProcessing
import re
import os
import hashlib
import time
import multiprocessing
import argparse
from datasets import load_from_disk
from lingua import Language, LanguageDetectorBuilder
import epitran
from functools import lru_cache
from transformers import PreTrainedTokenizerFast

In [2]:
dataset = load_dataset("wikitext", "wikitext-103-raw-v1")
dataset

DatasetDict({
    test: Dataset({
        features: ['text'],
        num_rows: 4358
    })
    train: Dataset({
        features: ['text'],
        num_rows: 1801350
    })
    validation: Dataset({
        features: ['text'],
        num_rows: 3760
    })
})

In [3]:
num_processes = multiprocessing.cpu_count() - 1
# Exact duplication removal (on individual sentences/paragraphs)
def remove_exact_duplicates(examples):
    seen = set()
    deduped_examples = []
    for sentence in examples["text"]:
        hash_val = hashlib.md5(sentence.encode()).hexdigest()
        if hash_val not in seen:
            seen.add(hash_val)
            deduped_examples.append(sentence)
    return {"text": deduped_examples}


def filter_by_language(examples):
    detector = LanguageDetectorBuilder.from_languages(Language.ENGLISH, Language.FRENCH).build()
    return {
        "text": [
            sentence for sentence in examples["text"] if detector.detect_language_of(sentence) == Language.ENGLISH
        ]
    }

In [4]:
# Basic text cleaning
def clean_text(examples):
    cleaned_text = []
    for sentence in examples["text"]:
        # Lowercase
        #sentence = sentence.lower()
        # Remove extra spaces
        sentence = re.sub(r"\s+", " ", sentence)
        # Remove URLs
        sentence = re.sub(r"http\S+", "", sentence)
        # Remove special characters
        sentence = re.sub(r"[^a-zA-Z0-9,.!?;:\'\" ]+", "", sentence)
        cleaned_text.append(sentence.strip())
    return {"text": cleaned_text}

def clean(dataset):
    dataset = dataset.map(remove_exact_duplicates, batched=True, num_proc=num_processes)
    dataset = dataset.map(filter_by_language, batched=True, num_proc=num_processes)
    dataset = dataset.map(clean_text, batched=True, num_proc=num_processes)
    return dataset


In [5]:
dataset_cleaned = dataset

In [6]:
dataset_cleaned

DatasetDict({
    test: Dataset({
        features: ['text'],
        num_rows: 4358
    })
    train: Dataset({
        features: ['text'],
        num_rows: 1801350
    })
    validation: Dataset({
        features: ['text'],
        num_rows: 3760
    })
})

In [7]:
epi = epitran.Epitran("eng-Latn")

@lru_cache(maxsize=None)
def xsampa_list(word: str) -> list:
    return epi.xsampa_list(word)

def translate_sentence(sentence: str) -> str:
    return ' '.join(' '.join(xsampa_list(word)) for word in sentence.split())

def translate_function(examples):
    return {"text": [translate_sentence(sentence) for sentence in examples["text"]]}


In [8]:
dataset_translated = dataset_cleaned.map(translate_function, batched=True, num_proc=num_processes)
dataset_translated

DatasetDict({
    test: Dataset({
        features: ['text'],
        num_rows: 4358
    })
    train: Dataset({
        features: ['text'],
        num_rows: 1801350
    })
    validation: Dataset({
        features: ['text'],
        num_rows: 3760
    })
})

In [9]:
dataset_translated["train"][:5]

{'text': ['',
  ' v { l k I r\\ i @ k r\\ A n @ k @ l z a j I i ',
  '',
  's E n dZ o n o w v { l k I r\\ i @   V n r\\ I k O r\\ d I d k r\\ A n @ k @ l z  dZ { p @ n i z    l I t  v { l k I r\\ i @ V v D @ b { t @ l f i l d    k A m @ n l i r\\ @ f r\\= d t @ { z v { l k I r\\ i @ k r\\ A n @ k @ l z a j I i a w t s a j d dZ @ p { n  I z @ t { k t I k @ l r\\ o w l  p l e j I N v I d i o w g e j m d I v E l @ p t b a j s i g @ { n d m i d i @ v I Z @ n f O r\\ D @ p l e j s t e j S @ n p O r\\ t @ b @ l  r\\ i l i s t I n dZ { n j u E r\\ i  I n dZ @ p { n  I t I z D @ T r\\= d g e j m I n D @ v { l k I r\\ i @ s I r\\ i z  E m p l o j I N D @ s e j m f j u Z @ n V v t { k t I k @ l { n d r\\ i l  t a j m g A m p l e j { z I t s p r\\ E d @ s E s r\\= z  D @ s t O r\\ i r\\ V n z p E r\\ @ l E l t @ D @ f r\\= s t g e j m { n d f A l o w z D @  n e j m l @ s   @ p i n @ l m I l @ t E r\\ i j u n @ t s r\\= v I N D @ n e j S @ n V v g { l i @ d U r\\ I N D @ s E k @ n d j U r\\ o w p

In [10]:
def get_training_corpus(dataset):
    dataset = concatenate_datasets([dataset["train"], dataset["test"], dataset["validation"]])
    for i in range(0, len(dataset), 1000):
        samples = dataset[i : i + 1000]
        yield samples["text"]

In [11]:
def train_tokenizer(dataset):
    tokenizer = Tokenizer(BPE(unk_token="[UNK]"))
    tokenizer.pre_tokenizer = Whitespace()
    tokenizer.post_processor = TemplateProcessing(
        single="[CLS] $A [SEP]",
        pair="[CLS] $A [SEP] $B:1 [SEP]:1",
        special_tokens=[
            ("[CLS]", 1),
            ("[SEP]", 2),
        ],
    )
    trainer = BpeTrainer(vocab_size=1000 ,special_tokens=["[PAD]", "[CLS]", "[SEP]", "[MASK]", "[UNK]"])
    tokenizer.train_from_iterator(get_training_corpus(dataset), trainer)
    tokenizer = PreTrainedTokenizerFast(tokenizer_object=tokenizer)
    tokenizer.add_special_tokens(
        {
            "pad_token": "[PAD]",
            "mask_token": "[MASK]",
            "cls_token": "[CLS]",
            "sep_token": "[SEP]",
            "unk_token": "[UNK]",
        }
    )
    return tokenizer

In [12]:
tokenizer = train_tokenizer(dataset_translated)

/home/toure215/miniconda3/envs/bert/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1617: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [13]:
print(tokenizer.vocab_size)
print(tokenizer.vocab)

159
{'u': 71, 'Z': 48, '[PAD]': 0, '_h': 102, 'o_X': 93, 'B': 25, '|\\|\\': 124, '|': 77, 'k_h': 138, 'F': 28, 'X_w': 148, 'p': 67, 'tS': 82, '_T': 107, 'K': 33, '[MASK]': 3, '3': 11, '=': 20, '[UNK]': 4, '}': 78, 'l_': 151, 'z_': 142, 'h': 59, 'w': 73, '!': 5, 'u_': 84, 'z': 75, '{': 76, 'n_G': 109, '_H': 121, 'S': 41, 'I': 31, 'e': 56, 'T': 42, '4': 12, '6': 14, '_w': 98, 'dZ': 81, '&': 7, 'l_G': 108, '`:': 149, 'u_X': 94, 'm_G': 100, 't_h': 154, '4_G': 92, 'R': 40, 'M': 35, 'b_': 116, 'j': 61, "\\'": 99, 'u_k': 156, '9': 17, 's_h': 140, 'V': 44, 't': 70, '?\\:': 130, 'c': 54, 'z_G': 129, '[SEP]': 2, 'o_': 83, "'=": 101, '>': 21, 'i_': 88, 'k_w': 127, 's': 69, 'g_w_h': 158, 'k': 62, '1': 9, '8': 16, '@\\': 125, 'G_w': 132, 'i_G': 95, 'b_h': 136, 'T_': 147, 'W': 45, '7': 15, 'r': 68, 'g': 58, '~': 79, 'B_G': 143, 'X': 46, 'P': 38, 'G': 29, 'K_': 144, 'R_w': 145, 'g_': 137, 'g_w': 119, '{:': 120, '|\\': 105, 'A': 24, 'U': 43, 'a_': 85, 'd_h': 126, 's_': 115, "`'": 135, 'D': 26, 'a': 52

In [14]:
tokenizer.save_pretrained("/home/toure215/BERT_phonetic/tokenizers/tokenizer_phonetic_WordLevel")

('/home/toure215/BERT_phonetic/tokenizers/tokenizer_phonetic_WordLevel/tokenizer_config.json',
 '/home/toure215/BERT_phonetic/tokenizers/tokenizer_phonetic_WordLevel/special_tokens_map.json',
 '/home/toure215/BERT_phonetic/tokenizers/tokenizer_phonetic_WordLevel/tokenizer.json')

In [15]:
def chunked_text(examples):
    chunked_text = []
    for sentence in examples["text"]:
        words = sentence.split()
        chunks = [words[i : i + 256] for i in range(0, len(words), 256)]
        chunked_text.extend([" ".join(chunk) for chunk in chunks])
    return {"text": chunked_text}

In [16]:
dataset_chunked = (
    dataset_translated.map(chunked_text, batched=True, num_proc=num_processes)
    .flatten_indices()
    .filter(lambda x: len(x["text"]) > 0)
)

dataset_chunked

DatasetDict({
    test: Dataset({
        features: ['text'],
        num_rows: 5292
    })
    train: Dataset({
        features: ['text'],
        num_rows: 2179886
    })
    validation: Dataset({
        features: ['text'],
        num_rows: 4577
    })
})

In [17]:
dataset_chunked.save_to_disk("/home/toure215/BERT_phonetic/DATASETS/phoneme_wikitext")

Saving the dataset (0/1 shards):   0%|          | 0/5292 [00:00<?, ? examples/s]

Saving the dataset (0/2 shards):   0%|          | 0/2179886 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/4577 [00:00<?, ? examples/s]

In [18]:
tokenizer = PreTrainedTokenizerFast.from_pretrained("/home/toure215/BERT_phonetic/tokenizers/tokenizer_phonetic_WordLevel")

In [19]:
s = translate_sentence("Hello, how are you?")
print(s)
e = tokenizer.encode(s)
d = tokenizer.decode(e)
print(e)
print(d)

h @ l o w h a w A r\ j u
[1, 59, 23, 63, 66, 73, 59, 52, 73, 24, 68, 49, 61, 71, 2]
[CLS] h @ l o w h a w A r \ j u [SEP]


In [20]:
print(dataset_chunked["train"][:5])

{'text': ['v { l k I r\\ i @ k r\\ A n @ k @ l z a j I i', 's E n dZ o n o w v { l k I r\\ i @ V n r\\ I k O r\\ d I d k r\\ A n @ k @ l z dZ { p @ n i z l I t v { l k I r\\ i @ V v D @ b { t @ l f i l d k A m @ n l i r\\ @ f r\\= d t @ { z v { l k I r\\ i @ k r\\ A n @ k @ l z a j I i a w t s a j d dZ @ p { n I z @ t { k t I k @ l r\\ o w l p l e j I N v I d i o w g e j m d I v E l @ p t b a j s i g @ { n d m i d i @ v I Z @ n f O r\\ D @ p l e j s t e j S @ n p O r\\ t @ b @ l r\\ i l i s t I n dZ { n j u E r\\ i I n dZ @ p { n I t I z D @ T r\\= d g e j m I n D @ v { l k I r\\ i @ s I r\\ i z E m p l o', 'j I N D @ s e j m f j u Z @ n V v t { k t I k @ l { n d r\\ i l t a j m g A m p l e j { z I t s p r\\ E d @ s E s r\\= z D @ s t O r\\ i r\\ V n z p E r\\ @ l E l t @ D @ f r\\= s t g e j m { n d f A l o w z D @ n e j m l @ s @ p i n @ l m I l @ t E r\\ i j u n @ t s r\\= v I N D @ n e j S @ n V v g { l i @ d U r\\ I N D @ s E k @ n d j U r\\ o w p { n w O r\\ h u p r\\= f O r\\ m 